In [48]:
import pandas as pd

In [49]:
import numpy as np

In [50]:
movies = pd.read_csv("./dataset/tmdb_5000_movies.csv")

In [51]:
credits = pd.read_csv("./dataset/tmdb_5000_credits.csv")

In [52]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [53]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [54]:
movies.shape

(4803, 20)

In [55]:
credits.shape

(4803, 4)

In [56]:
movies = movies.merge(credits, on="title")

In [57]:
movies.shape

(4809, 23)

In [58]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [59]:
# genre
# id
# keyword
# title
# overview
# cast
# crew
movies = movies[["movie_id", "overview", "title", "genres", "keywords", "cast", "crew"]]

In [60]:
movies.dropna(inplace=True)

In [61]:
movies.duplicated().sum()

0

In [62]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [69]:
from helper.extract import extract

ImportError: cannot import name 'extract_cast' from 'helper.extract' (/home/marco/marco/projects/movie_recommender/helper/extract.py)

In [64]:
movies["keywords"] = movies["keywords"].apply(extract("name"))
movies["genres"] = movies["genres"].apply(extract("name"))

In [74]:
movies.head(1)

,movie_id,overview,title,genres,keywords,cast,crew
0,19995,"In the 22nd century, a paraplegic Marine is di...",Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [77]:
from ast import literal_eval
def extract_cast(obj):
    lst: List[str] = []
    counter = 0
    for i in literal_eval(obj):
        if counter != 3:
            lst.append(i["name"])
            counter += 1
        else:
            break
    return lst

In [ ]:
movies["cast"] = movies["cast"].apply(extract_cast)

In [ ]:
movies.head(1)

In [81]:
def get_director(obj):
    lst: List[str] = []
    for i in literal_eval(obj):
        if i["job"] == "Director":
            lst.append(i["name"])
            break  # if found, break
    return lst

In [82]:
movies["crew"] = movies["crew"].apply(get_director)

In [ ]:
movies.head(1)

In [83]:
movies["overview"] = movies["overview"].apply(lambda obj: obj.split())

In [84]:
movies["genres"] = movies["genres"].apply(lambda obj: [i.replace(" ", "") for i in obj])
movies["keywords"] = movies["keywords"].apply(lambda obj: [i.replace(" ", "") for i in obj])
movies["cast"] = movies["cast"].apply(lambda obj: [i.replace(" ", "") for i in obj])
movies["crew"] = movies["crew"].apply(lambda obj: [i.replace(" ", "") for i in obj])

In [ ]:
movies.head(1)

In [85]:
movies["tags"] = movies["overview"] + movies["genres"] + movies["keywords"] + movies["cast"] + movies["crew"]

In [86]:
new_df = movies[["movie_id", "title", "tags"]]

In [87]:
new_df["tags"] = movies["tags"].apply(lambda obj: " ".join(obj))

/tmp/ipykernel_23856/998153284.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = movies["tags"].apply(lambda obj: " ".join(obj))


In [88]:
new_df["tags"] = new_df["tags"].apply(lambda obj: obj.lower())

/tmp/ipykernel_23856/351979775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(lambda obj: obj.lower())


In [89]:
new_df

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."
...,...,...,...
4804,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,a newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""signed, sealed, delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [90]:
new_df["tags"]

0       in the 22nd century, a paraplegic marine is di...
1       captain barbossa, long believed to be dead, ha...
2       a cryptic message from bond’s past sends him o...
3       following the death of district attorney harve...
4       john carter is a war-weary, former military ca...
                              ...                        
4804    el mariachi just wants to play his guitar and ...
4805    a newlywed couple's honeymoon is upended by th...
4806    "signed, sealed, delivered" introduces a dedic...
4807    when ambitious new york attorney sam is sent t...
4808    ever since the second grade when he first saw ...
Name: tags, Length: 4806, dtype: object

In [91]:
from sklearn.feature_extraction.text import CountVectorizer

In [92]:
cv = CountVectorizer(max_features=5000, stop_words="english")

In [93]:
vector = cv.fit_transform(new_df["tags"]).toarray()

In [94]:
from nltk.stem.porter import PorterStemmer

In [95]:
ps = PorterStemmer()

In [96]:
def stem(text: str) -> str:
    return " ".join([ ps.stem(i) for i in text.split() ])

In [97]:
new_df["tags"] = new_df["tags"].apply(stem)

/tmp/ipykernel_23856/825579503.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df["tags"] = new_df["tags"].apply(stem)


In [98]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
sorted(list(enumerate(similarity[0])), reverse=True, key=lambda x: x[1])[1:6]

In [ ]:
def recommend(movie: str):
    idx = new_df[new_df["title"] == movie].index[0]
    dst = similarity[idx]
    lst = sorted(list(enumerate(dst)), reverse=True, key=lambda x: x[1])[1:6]
    for i in lst:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend("Iron Man")

In [ ]:
import pickle

In [ ]:
pickle.dump(new_df, open("movies.pkl", "wb"))

In [ ]:
pickle.dump(similarity, open("similarity.pkl", "wb"))

In [ ]:
pickle.dump(new_df.to_dict(), open("movies_dict.pkl", "wb"))